In [147]:
import pandas as pd
import numpy as np

## Answer key cleaning/setup

In [115]:
#answer key cleaning/setup

# answer_key = pd.read_csv("raw_data/answers-answerkey.csv")
# answer_key = answer_key.drop(columns=["time", "version"])
# answer_key.to_csv("raw_data/answerKey.csv", index=False)
answer_key = pd.read_csv("./answerKey.csv")
# answer_key = answer_key.drop(columns=["time", "version"])
# answer_key = answer_key[(answer_key["pageName"] != "Background Questions") & (answer_key["pageName"] != "SUS")]
# answer_key.to_csv("./answerKey.csv", index=False)
answer_key.head()

,pageName,question,answer
0,T1_S2_IN1,qID-1,Yes
1,T1_S2_IN1,qID-0,Continue
2,T1_S2_IN1,qID-11,Yes
3,T1_S2_IN1,qID-12,Step 8
4,T1_S2_IN1,qID-13,Open-ended Correct


In [116]:
OPEN_CR = "Open-ended Correct"
OPEN_PRCR = "Open-ended Partial"
OPEN_INCR = "Open-ended Incorrect"
proofs = {
    "T1_S1_C1" : OPEN_CR, 
    "T1_S1_C2" : OPEN_CR, 
    "T1_S1_IN1": OPEN_CR, 
    "T1_S1_IN2": OPEN_CR, 
    "T1_S1_IN3" :OPEN_CR, 
    "T1_S2_C1": OPEN_CR,
    "T1_S2_C2": OPEN_CR,
    "T1_S2_IN1": OPEN_CR,
    "T1_S2_IN2": OPEN_CR,
}
def update_openended(df, proofs):
    for k, v in proofs.items():
        df.loc[(df["pageName"] == k) & (df["question"] == "qID-13"), "answer"] = v
    return df
answer_key = update_openended(answer_key, proofs)
answer_key.head()

,pageName,question,answer
0,T1_S2_IN1,qID-1,Yes
1,T1_S2_IN1,qID-0,Continue
2,T1_S2_IN1,qID-11,Yes
3,T1_S2_IN1,qID-12,Step 8
4,T1_S2_IN1,qID-13,Open-ended Correct


## Grading Individual Tests

In [414]:
participant = "pD"
isPilot = True

In [415]:
folder = "pilot-data" if isPilot else "study-data"
answers_df = pd.read_csv(f'{folder}/answers-{participant}.csv')
event_logs_df = pd.read_csv(f'{folder}/eventLogs-{participant}.csv')

In [416]:
# scoring question where student explains how to correct
CR = "Open-ended Correct"
INCR = "Open-ended Incorrect"
proofs = {
    "T1_S1_C1" : CR, 
    "T1_S1_C2" : CR, 
    "T1_S1_IN1": CR, 
    "T1_S1_IN2": INCR, 
    "T1_S1_IN3" : INCR, 
    "T1_S2_C2": CR,
    "T1_S2_IN1": CR,
    "T1_S2_IN2": CR,
}

In [417]:
event_logs_df.rename(columns={'t': 'time'}, inplace=True)
event_logs_df.rename(columns={'e': 'event'}, inplace=True)

# Convert the 'time' columns to datetime format
answers_df['time'] = pd.to_datetime(answers_df['time'], unit='ms')
event_logs_df['time'] = pd.to_datetime(event_logs_df['time'], unit='ms')

# Sort both DataFrames by the 'time' column
answers_df = answers_df.sort_values(by='time')
event_logs_df = event_logs_df.sort_values(by='time')

#reindex
answers_df = answers_df.reset_index(drop=True)
event_logs_df = event_logs_df.reset_index(drop=True)

#Drop unused, stale answers to questions. Anything that occurred before background questions
background_index = answers_df[answers_df['pageName'] == 'Background Questions'].index[0]
answers_df = answers_df.loc[answers_df.index >= background_index]

#reindex again
answers_df = answers_df.reset_index(drop=True)

# Merge the DataFrames based on the 'time' column
# merged_df = pd.concat([answers_df, event_logs_df]).sort_values(by='time').reset_index(drop=True)

# Replace values in the 'type' column
answers_df['version'] = answers_df['version'].replace({'static': 'B', 'interactive': 'A'})

# update scores for open-ended problems
update_openended(answers_df, proofs)

# Display the first few rows of the merged DataFrame
answers_df.head(30)

,pageName,question,answer,time,version
0,Background Questions,0,17,2024-12-08 20:11:46.071,undefined
1,Background Questions,1,10th grade,2024-12-08 20:11:46.071,undefined
2,Background Questions,2,11th grade,2024-12-08 20:11:46.071,undefined
3,Background Questions,3,regular,2024-12-08 20:11:46.071,undefined
4,P1,qID-2,MN,2024-12-08 20:13:48.550,B
5,P1,qID-1,BC,2024-12-08 20:14:13.448,B
6,P1,qID-3,HJ,2024-12-08 20:14:40.032,B
7,P2,qID-1,Angle GHJ,2024-12-08 20:15:15.751,B
8,P2,qID-3,Angle NQP,2024-12-08 20:15:28.355,B
9,P2,qID-2,Angle DEF,2024-12-08 20:15:52.117,B


In [418]:
# Score the test
scores, aks = [], []
valid_ids = set(["qID-11", "qID-12", "qID-13"])
correct_proof_ids = set(["T1_S1_C1", "T1_S1_C2", "T1_S2_C2"])
incorrect_proof_ids = set(["T1_S1_C1", "T1_S1_IN2", "T1_S1_IN3", "T1_S2_IN1","T1_S2_IN2"])

for index, row in answers_df.iterrows():
    question, proof = row['question'], row['pageName']
    a = row.loc['answer']

    # page is not included in the answer key
    if not proof in set(answer_key['pageName']) or proof.startswith("Tutorial"):
        scores.append(None)
        aks.append(None)
        continue
    
    #special case for pretest where some questions are inserted at the 1st question about triangle congruence
    if proof.startswith("P") and question in valid_ids:
        ans_row = answer_key.loc[(answer_key.pageName=="P5") & (answer_key.question==question)]
    else:
        # find the proof and the question being scored in answer key
        ans_row = answer_key.loc[(answer_key.question==question) & (answer_key.pageName==proof)]
        
    # this question/proof combination is not in the answer key
    if len(ans_row) == 0:
        scores.append(0)
        aks.append(None)
        continue

    if proof in correct_proof_ids and question == "qID-11" and a == "No":
        #student correctly said there was no mistake 
        scores.append(3)
    else:
        # add score to list
        correct = ans_row['answer'].values[0] == a
        scores.append(1 if correct else 0)
    aks.append(ans_row['answer'].values[0])

# add columns to answer dataframe
answers_df["score"] = pd.Series(scores).values
answers_df["key"] = pd.Series(aks).values
answers_df.head(60)

,pageName,question,answer,time,version,score,key
0,Background Questions,0,17,2024-12-08 20:11:46.071,undefined,NaN,None
1,Background Questions,1,10th grade,2024-12-08 20:11:46.071,undefined,NaN,None
2,Background Questions,2,11th grade,2024-12-08 20:11:46.071,undefined,NaN,None
3,Background Questions,3,regular,2024-12-08 20:11:46.071,undefined,NaN,None
4,P1,qID-2,MN,2024-12-08 20:13:48.550,B,1.0,MN
5,P1,qID-1,BC,2024-12-08 20:14:13.448,B,1.0,BC
6,P1,qID-3,HJ,2024-12-08 20:14:40.032,B,1.0,HJ
7,P2,qID-1,Angle GHJ,2024-12-08 20:15:15.751,B,1.0,Angle GHJ
8,P2,qID-3,Angle NQP,2024-12-08 20:15:28.355,B,1.0,Angle NQP
9,P2,qID-2,Angle DEF,2024-12-08 20:15:52.117,B,1.0,Angle DEF


In [419]:
# add time elapsed column
answers_df['delta'] = answers_df['time'].diff().dt.total_seconds().fillna(0)

# add participant id column
answers_df["participant"] = pd.Series([participant for i in range(len(answers_df))]).values
answers_df.head()

,pageName,question,answer,time,version,score,key,delta,participant
0,Background Questions,0,17,2024-12-08 20:11:46.071,undefined,NaN,None,0.000,pD
1,Background Questions,1,10th grade,2024-12-08 20:11:46.071,undefined,NaN,None,0.000,pD
2,Background Questions,2,11th grade,2024-12-08 20:11:46.071,undefined,NaN,None,0.000,pD
3,Background Questions,3,regular,2024-12-08 20:11:46.071,undefined,NaN,None,0.000,pD
4,P1,qID-2,MN,2024-12-08 20:13:48.550,B,1.0,MN,122.479,pD


In [420]:
#store the combined CSV
answers_df.to_csv(f"./study-data/processed/{participant}.csv", index=False)

In [421]:
def separate_data(df, page):
    # Create a new DataFrame with rows where pageName == page
    df_new = df[df['pageName'] == page]

    # Remove those rows from the original DataFrame
    df = df[df['pageName'] != page]

    return df, df_new
# separate out the background questions to new df
answers_df, df_back = separate_data(answers_df, "Background Questions")

# separate the SUS scores into a new df
answers_df, df_sus = separate_data(answers_df, "SUS")
df_sus = df_sus.drop(columns=["time", "version"])

In [422]:
# score the participant by pretest, activity, SUS
# max score for pretest
mask = lambda df: df[df["pageName"].str.startswith('P')]
pretest_max_score = len(mask(answer_key))
#score pretest
pre = mask(answers_df)
pre_score = pre["score"].sum()

# max score for activity
mask = lambda df: df[~df['pageName'].str.startswith('P')]
activity_df = mask(answer_key)
#score activity
act = mask(answers_df)
act_score = act["score"].sum()

# when proof is correct, add 2 to score (because the "which step is wrong?" and "explain how to correct" questions are skipped)
# There are 3 correct proofs so add 6 points to total for the answer key
activity_max_score = len(activity_df) + 6

# calculate sus score
df_sus["score"] = np.zeros(len(df_sus))
for idx, row in df_sus.iterrows():
    # question indices are 0-indexed
    q, a = int(row["question"]), int(row["answer"])
    df_sus.loc[(df_sus["question"] == row["question"]),"score"] = (a - 1) if q % 2 == 0 else (5-a)
sus_score = df_sus["score"].sum() * 2.5

print(f"pretest: {pre_score}/{pretest_max_score},\nactivity: {act_score}/{activity_max_score},\nsus: {sus_score}")

pretest: 14.0/14,
activity: 42.0/51,
sus: 75.0


## Concatenate Individual Participants Scores
Create SUS, Scores, Background CSVs for all participant

In [423]:
# dataframe to collect scores only, 1 new row per participant, include boolean flag if the data came from a pilot
def add_participant_totals(participant, answers_df, overwrite=False):
    filename = "./out/study/scores_df.csv"
    df = pd.read_csv(filename)
    row = pd.DataFrame({
        "id": participant, 
        "version": answers_df.loc[(answers_df["pageName"] == "T1_S1_C1") & (answers_df["question"] == "qID-0"),"version"].values[0], 
        "sus":sus_score, 
        "pretest": float("%.2f" % round(pre_score / pretest_max_score, 2)), 
        "score": float("%.2f" % round(act_score / activity_max_score, 2)), 
        "pilot": 1 if isPilot else 0
    }, index=[0])
    if df[df["id"] == participant].empty:
        df = pd.concat([df, row], ignore_index=True)
        df = df.drop_duplicates()
    elif overwrite:
        df.loc[df["id"] == participant] = row
    df.to_csv(filename, index=False)
    return df

df = add_participant_totals(participant, answers_df, overwrite=True)
df

,id,version,sus,pretest,score,pilot
0,elephant,A,95.0,1.00,0.80,0
1,penguin,B,87.5,0.79,0.45,0
2,peacock,A,87.5,0.71,0.84,0
3,eagle,B,100.0,0.86,0.35,0
4,giraffe,A,90.0,0.79,0.67,0
5,hippo,B,75.0,0.79,0.51,0
6,rhino,A,70.0,0.79,0.51,0
7,pA,A,80.0,0.93,0.76,1
8,pB,A,95.0,0.93,0.88,1
9,pC,B,87.5,0.71,0.69,1


In [424]:
def add_participant_timing(participant, answers_df, overwrite=False):
    filename = "./out/study/per_question.csv"
    df = pd.read_csv(filename)

    #convert answers of participant into appropriate format
    answers_df = answers_df.rename(columns={'participant': 'id'})
    answers_df = answers_df.rename(columns={'pageName': 'proof'})
    answers_df = answers_df.rename(columns={'delta': 'time_elapsed'})
    answers_df = answers_df.drop(columns=["key", "answer", "time"])
    answers_df = answers_df[(answers_df.proof != "Background Questions") & (answers_df.proof != "SUS") & (~answers_df.proof.str.startswith("Tutorial"))]
    answers_df["pilot"] = [1 if isPilot else 0 for x in range(len(answers_df))]
    if overwrite:
        df = df[df["id"] != participant]
    if df[df["id"] == participant].empty:
        df = pd.concat([df, answers_df], ignore_index=True)
        df = df.drop_duplicates()

    df = df.reset_index(drop=True)
    df.to_csv(filename, index=False)
    return df
df = add_participant_timing(participant, answers_df, overwrite=False)
df

,id,question,version,time_elapsed,proof,score,pilot
0,elephant,qID-2,B,82.740,P1,1.0,0.0
1,elephant,qID-3,B,10.533,P1,1.0,0.0
2,elephant,qID-1,B,9.354,P1,1.0,0.0
3,elephant,qID-3,B,8.984,P2,1.0,0.0
4,elephant,qID-2,B,8.384,P2,1.0,0.0
...,...,...,...,...,...,...,...
562,pD,qID-11,A,3.367,T1_S2_IN2,1.0,1.0
563,pD,qID-12,A,6.219,T1_S2_IN2,1.0,1.0
564,pD,qID-13,A,25.218,T1_S2_IN2,1.0,1.0
565,pD,qID-2,A,42.416,T1_S2_IN2,1.0,1.0


## Graveyard

In [174]:
# # Add a new column to answers_df to indicate the next answer time
# answers_df['next_time'] = answers_df['time'].shift(-1)

# # Merge each event log with the corresponding answer based on time interval
# merged_df = pd.merge_asof(event_logs_df, answers_df, on='time', direction='backward')

# event_count_dict = {}

# # Define event types
# event_types = ['n', 'c', 'h', 'm', 'p']

# # Iterate through each answer to count events between time intervals
# for index, row in answers_df.iterrows():
#     answer_time = row['time']
#     next_time = row['next_time'] if not pd.isnull(row['next_time']) else pd.Timestamp.max
    
#     # Select events that fall within the time interval
#     filtered_events = merged_df[(merged_df['time'] >= answer_time) & (merged_df['time'] < next_time)]
    
#     # Filter out "hover" events (h) that occur within 0.01 second of another event [debounce]
#     filtered_events = filtered_events[~((filtered_events['event'] == 'h') &
#                                        (filtered_events['time'].diff().dt.total_seconds() <= 0.01))]
    
#     # Count occurrences of each event type
#     event_counts = filtered_events['event'].value_counts()
    
#     # Initialize counts for all event types
#     counts = {event: event_counts.get(event, 0) for event in event_types}
    
#     # Store the counts in the dictionary
#     event_count_dict[answer_time] = counts

# # Convert the event count dictionary into a DataFrame
# event_counts_df = pd.DataFrame.from_dict(event_count_dict, orient='index').fillna(0)

# # Join the event counts with the answers_df
# final_df = pd.concat([answers_df.set_index('time'), event_counts_df], axis=1).reset_index()

# # Drop the next_time column as it's no longer needed
# final_df = final_df.drop(columns=['next_time'])

# # rename version to condition column
# final_df = final_df.rename(columns={"version" : "condition"})

# # Rename columns
# final_df.rename(columns={'n': 'next'}, inplace=True)
# final_df.rename(columns={'c': 'click'}, inplace=True)
# final_df.rename(columns={'m': 'mouse'}, inplace=True)
# final_df.rename(columns={'h': 'hover'}, inplace=True)
# final_df.rename(columns={'p': 'pointer'}, inplace=True)


# # Display the first few rows of the final DataFrame
# final_df.head(30)